In [1]:
import os
path = os.path.normpath(os.getcwd() + os.sep + os.pardir)
os.chdir(path)
path

'E:\\Semester_10\\IE506\\KFSC-Python'

In [2]:
import time
from multiprocessing import Pool
import multiprocessing
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import accuracy_score
from BestMap import BestMap
from KFSC import KFSC
from KFSC_LARGE import KFSC_LARGE
from KFSC_minibatch import KFSC_MB
from multi_tester import test_dataset


Bad key "text.kerning_factor" on line 4 in
C:\Users\anura\anaconda3\envs\env_full\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
dataset = 'mnist_sc_f150.mat'
# dataset = 'fmnist_fea_150.mat'
# dataset = 'Epileptic.mat'
f = sio.loadmat(dataset)
X = f['X']
label = np.concatenate(f['Label'])
k = len(np.unique(label))
cpu_count = multiprocessing.cpu_count()
use_cpu = 6
print("CPU count: ", cpu_count)
print("Using CPU: ", use_cpu)

CPU count:  8
Using CPU:  6


In [4]:
iterations = 30
funct_to_run = KFSC_MB
params = {}
params["d"] = np.linspace(4, 100, iterations, dtype=int)

In [5]:
[] + [X, k, label, funct_to_run, dataset]

[array([[-0.00387764, -0.00383456, -0.00388697, ..., -0.00381669,
         -0.0038033 , -0.00385804],
        [ 0.00496426,  0.00481199,  0.00382726, ...,  0.0016532 ,
          0.002295  ,  0.00102705],
        [ 0.00201634,  0.00390121, -0.00173864, ...,  0.00275295,
          0.00379308,  0.00297695],
        ...,
        [ 0.00123876, -0.00027094, -0.00446999, ...,  0.00798547,
         -0.00255349,  0.00164713],
        [-0.00111287,  0.0035701 ,  0.00267775, ..., -0.00352793,
         -0.00591071, -0.00205644],
        [-0.00151471, -0.00544073,  0.00094182, ..., -0.0024999 ,
         -0.00646517, -0.00016609]]),
 10,
 array([0, 0, 0, ..., 9, 9, 9], dtype=uint8),
 <function KFSC_minibatch.KFSC_MB(X, k, d, lamda, options: Optional[dict] = None)>,
 'mnist_sc_f150.mat']

In [6]:
arg_list = []
for i in range(iterations):
    arg_list.append([])
    arg_list[-1] += [X, k, label, funct_to_run, dataset]
    print(arg_list[-1])
    build_dict = {}
    for key in params.keys():
        build_dict[key] = params[key][i]
    arg_list[-1].append(build_dict)
    arg_list[-1] = tuple(arg_list[-1])
arg_list[23]

[array([[-0.00387764, -0.00383456, -0.00388697, ..., -0.00381669,
        -0.0038033 , -0.00385804],
       [ 0.00496426,  0.00481199,  0.00382726, ...,  0.0016532 ,
         0.002295  ,  0.00102705],
       [ 0.00201634,  0.00390121, -0.00173864, ...,  0.00275295,
         0.00379308,  0.00297695],
       ...,
       [ 0.00123876, -0.00027094, -0.00446999, ...,  0.00798547,
        -0.00255349,  0.00164713],
       [-0.00111287,  0.0035701 ,  0.00267775, ..., -0.00352793,
        -0.00591071, -0.00205644],
       [-0.00151471, -0.00544073,  0.00094182, ..., -0.0024999 ,
        -0.00646517, -0.00016609]]), 10, array([0, 0, 0, ..., 9, 9, 9], dtype=uint8), <function KFSC_MB at 0x0000022E90E8B4C0>, 'mnist_sc_f150.mat']
[array([[-0.00387764, -0.00383456, -0.00388697, ..., -0.00381669,
        -0.0038033 , -0.00385804],
       [ 0.00496426,  0.00481199,  0.00382726, ...,  0.0016532 ,
         0.002295  ,  0.00102705],
       [ 0.00201634,  0.00390121, -0.00173864, ...,  0.00275295,
       

(array([[-0.00387764, -0.00383456, -0.00388697, ..., -0.00381669,
         -0.0038033 , -0.00385804],
        [ 0.00496426,  0.00481199,  0.00382726, ...,  0.0016532 ,
          0.002295  ,  0.00102705],
        [ 0.00201634,  0.00390121, -0.00173864, ...,  0.00275295,
          0.00379308,  0.00297695],
        ...,
        [ 0.00123876, -0.00027094, -0.00446999, ...,  0.00798547,
         -0.00255349,  0.00164713],
        [-0.00111287,  0.0035701 ,  0.00267775, ..., -0.00352793,
         -0.00591071, -0.00205644],
        [-0.00151471, -0.00544073,  0.00094182, ..., -0.0024999 ,
         -0.00646517, -0.00016609]]),
 10,
 array([0, 0, 0, ..., 9, 9, 9], dtype=uint8),
 <function KFSC_minibatch.KFSC_MB(X, k, d, lamda, options: Optional[dict] = None)>,
 'mnist_sc_f150.mat',
 {'d': 80})

In [ ]:
with Pool(processes=use_cpu) as p:
    df_list = p.starmap(test_dataset, arg_list)
combined_df = pd.concat(df_list, ignore_index=True)
path = os.getcwd()
os.chdir(path+'//Scores')
combined_df.to_csv(dataset.split('_')[0] + 'run_info.csv', mode='a', index=False, header=False)